In [74]:
from langchain_cohere import ChatCohere
from langchain.llms import Cohere
import os
from langchain.docstore.document import Document
from pdfminer.high_level import extract_text
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

In [75]:
load_dotenv()

COHERE_API_KEY = os.getenv("COHERE_API_KEY")
llm = ChatCohere(cohere_api_key=COHERE_API_KEY)
llm2 = Cohere(cohere_api_key=COHERE_API_KEY)

In [10]:
answer = llm2(prompt="What is the capital of France?")
print(answer)

/tmp/ipykernel_6039/2087365494.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  answer = llm2(prompt="What is the capital of France?")


 The capital of France is Paris. Paris is France's largest city and a major global center for art, culture, fashion, and cuisine. It is renowned for its iconic landmarks, such as the Eiffel Tower, the Louvre Museum, and the Arc de Triomphe, as well as for its picturesque streets and neighborhoods, including the Champs-Élysées and Montmartre. 

As the capital, Paris serves as the political and administrative center of France. It is home to important government buildings and institutions, including the Palais Garnier, the seat of the French National Assembly, and the Palais Bourbon, which houses the French Senate. 

Beyond its administrative and cultural importance, Paris is a major hub for commerce, tourism, and education. It hosts the headquarters of many multinational corporations and is a popular destination for tourists from around the world. 

The city is also home to several prestigious universities and colleges, such as the Sorbonne and the École Normale Supérieure, making it an 

In [66]:
pdf_text = extract_text("./images/Resume.pdf")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_text(pdf_text)

cohere_embeddings = CohereEmbeddings(model= "embed-english-v3.0",cohere_api_key=COHERE_API_KEY)

documents = [Document(page_content=chunk) for chunk in chunks]

vector_store = FAISS.from_documents(documents, cohere_embeddings)


In [76]:
retriever = vector_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

In [78]:
query = "Where did she work before?"
response = qa_chain.run(query)
print(response)

Eniko Kakas has worked at the following places:

- Ecogoggle BV, Eindhoven, as a Data Science Intern from February 2024 to July 2024.
- Calima Solutions SL, Las Palmas de Gran Canaria, as a Software Developer Intern from August 2022 to January 2023.
- ISSD Fontys ICT, Eindhoven, as a Student Assistant from October 2021 to February 2024.
